In [1]:
import sys
import pandas as pd
import os
from path import Path
import plotly
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from urllib.request import urlopen
import json
import statsmodels as sm
from statsmodels.formula.api import ols
from Bio import Seq, SeqIO, AlignIO, Phylo, Align
from jinja2 import Environment, FileSystemLoader  # html template engine
# import cv2
import numpy as np
import skimage as sk
import matplotlib.pylab as plt
import datetime as dt

In [2]:
sys.path.append('../')

In [3]:
import bjorn_support as bs
import onion_trees as ot
import mutations as bm
import visualize as bv
import reports as br

In [4]:
msa_fp = '/home/al/analysis/gisaid/sequences_2021-01-25_08-14_aligned.fasta.gz'
meta_fp = '/home/al/analysis/gisaid/metadata_2021-01-25_09-11.tsv.gz'

In [5]:
cols = {'Virus name': 'strain', 'Collection date': 'date' , 'Additional host information': 'purpose_of_sequencing',
        'Lineage': 'pangolin_lineage', 'Host': 'host'}
old_meta_fp = meta_fp
meta_fp = '/home/al/analysis/gisaid/metadata_2021-01-26.tsv.gz'
xtra_fp = '/home/al/analysis/gisaid/gisaid_hcov-19_2021_01_26_19.tsv'
drop_cols = ['Accession ID', 'Location', 'Passage', 'Specimen', 'Sequencing technology',
             'Assembly method', 'Comment', 'Comment type']

In [6]:
# meta = bs.integrate_gisaid_meta(old_meta_fp, xtra_fp, msa_fp, cols, drop_cols)

In [7]:
meta_fp

'/home/al/analysis/gisaid/metadata_2021-01-26.tsv.gz'

In [8]:
# meta.to_csv(meta_fp, sep='\t', compression='gzip', index=False)

In [24]:
# meta = pd.read_csv(meta_fp, sep='\t', compression='gzip')
# meta.loc[(meta['country'].str.contains('United States'))&(meta['pangolin_lineage']=='B.1.1.7')]

/home/al/anaconda3/envs/bjorn/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning:

Columns (8,28) have mixed types.Specify dtype option on import or set low_memory=False.



In [10]:
# gisaid, _ = bm.identify_replacements_per_sample(msa_fp, meta_fp, bm.GENE2POS, data_src='gisaid', test=False)
# gisaid['mutation'] = gisaid['gene'] + ':' + gisaid['ref_aa'] + gisaid['codon_num'].astype(str) + gisaid['alt_aa']
# gisaid.loc[gisaid['country']=='USA', 'country'] = 'United States of America'

In [11]:
# gisaid_fp = '/home/al/analysis/gisaid/subs_long_2021-01-15_14-55v2.csv.gz'
# gisaid.to_csv(gisaid_fp, index=False, compression='gzip')

In [12]:
# gisaid[(gisaid['mutation']=='ORF1b:D1183Y')]['strain'].unique().shape

In [8]:
# feature = 'mutations'
# values = ['S:S13I', 'S:W152C', 'S:L452R']
feature = 'pangolin_lineage'
# S:Q677H, M:A85S, N:D377Y,
values = ['B.1.1.7']#, 'S:W152C', 'S:L452R'] # 'ORF1ab:I4205V', 
input_params = {
    'gisaid_data_fp' : '/home/al/analysis/gisaid/subs_long_2021-01-25.csv.gz',
    'gisaid_meta_fp': meta_fp,
    'vocs': ['B.1.1.7', 'B.1.1.70'],
    'strain': 'B117',
    'date': '01/26/2021',
    'msa_fp': Path(msa_fp),
    'meta_fp' : Path('/home/al/code/HCoV-19-Genomics/metadata.csv'),
    'tree_fp' : Path('/home/al/analysis/alab_mutations_01-01-2021/alab/seqs_aligned.fa.treefile'),
    'subs_fp' : '/home/al/analysis/alab_mutations_01-01-2021/alab_substitutions_long_01-01-2021.csv',
    'countries_fp' : '/home/al/data/geojsons/countries.geo.json',
    'states_fp' : "/home/al/data/geojsons/us-states.json",
    'counties_fp' : '/home/al/data/geojsons/us-counties.json',
    'patient_zero' : 'NC_045512.2',
    'b117_meta' : '/home/al/analysis/b117/nextstrain_groups_neherlab_ncov_S.N501_metadata.tsv',
    'b117_tree': 'test_data/b117_seqs_aligned.fasta.treefile',
    'sample_sz': 500,
    'sampling_img_fp' : "/home/al/analysis/b117/figs/sars-cov-2_EM_v3.png"
}

In [9]:
results = br.generate_voc_data(feature, values, input_params)

/home/al/anaconda3/envs/bjorn/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3338: DtypeWarning: Columns (8,28) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [10]:
html = br.generate_voc_html(feature, values, results, template_name='voc.html')
# br.save_html(html, f'test_data/orf1ab_i4205v_report.html')
br.save_html(html, f'test_data/b117_current_report.html')

In [ ]:
plotly.offline.plot()

In [17]:
gisaid = pd.read_csv(input_params['gisaid_data_fp'], compression='gzip')

In [16]:
# res = (gisaid.groupby(['date', 'country', 'division', 
#                                         'purpose_of_sequencing',
#                                         'location', 'pangolin_lineage', 'strain'])
#                        .agg(mutations=('mutation', 'unique')).reset_index())
# res['is_vui'] = res['mutations'].apply(bv.is_vui, args=(set(values),))

In [16]:
gisaid = pd.read_csv(input_params['gisaid_data_fp'], compression='gzip')

In [12]:

def get_mutations(data: pd.DataFrame, lineage: str='B.1.1.7'):
    mutations = set(data[data['pangolin_lineage']==lineage]['mutation'].unique().tolist())
    return mutations

In [12]:
# gisaid = pd.read_csv(input_params['gisaid_data_fp'], compression='gzip')
# gisaid.drop(columns=['mutation'], inplace=True)
# gisaid['mutations'] = gisaid['gene'] + ':' + gisaid['ref_aa'] + gisaid['codon_num'].astype(str) + gisaid['alt_aa']
# gisaid.to_csv(input_params['gisaid_data_fp'], compression='gzip')

/home/al/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (38,39,41,42,43,44,45,46) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [20]:
gisaid[gisaid['pangolin_lineage']=='B.1.1.7']['strain'].unique().shape

(10421,)

In [21]:
gisaid[gisaid['pangolin_lineage']=='B.1.1.70']['strain'].unique().shape

(1340,)

In [18]:
b117 = get_mutations(gisaid, lineage='B.1.1.7')
b1170 = get_mutations(gisaid, lineage='B.1.1.70')
b117.intersection(b1170)

{'3UTR:A15T',
 '3UTR:G35G',
 '3UTR:I4I',
 '3UTR:Q66K',
 '3UTR:T32I',
 '3UTR:V46L',
 '3UTR:V6L',
 '3UTR:_31L',
 '3UTR:_9W',
 '5UTR:I74I',
 '5UTR:P73P',
 '5UTR:R81C',
 '5UTR:S68L',
 'E:F23F',
 'E:L65L',
 'E:L73F',
 'E:S68F',
 'E:T30I',
 'M:D160D',
 'M:D190D',
 'M:F112F',
 'M:F53F',
 'M:G202G',
 'M:G78C',
 'M:L124L',
 'M:L35L',
 'M:N203N',
 'M:N41N',
 'M:S173K',
 'M:T172T',
 'M:Y196Y',
 'M:Y71Y',
 'N:A134V',
 'N:A156S',
 'N:A208S',
 'N:A211S',
 'N:A252S',
 'N:A273S',
 'N:A359S',
 'N:A35A',
 'N:A90S',
 'N:D348Y',
 'N:D401Y',
 'N:D402Y',
 'N:D415D',
 'N:F110F',
 'N:F274F',
 'N:F346F',
 'N:G204R',
 'N:H145H',
 'N:I292I',
 'N:I337I',
 'N:N126N',
 'N:N192N',
 'N:P142S',
 'N:P199S',
 'N:P383P',
 'N:Q28H',
 'N:Q418H',
 'N:Q9H',
 'N:R195I',
 'N:R203K',
 'N:R385I',
 'N:S194P',
 'N:S413I',
 'N:T135I',
 'N:T16M',
 'N:T16T',
 'N:T24I',
 'N:T282T',
 'N:T379I',
 'N:T417I',
 'N:T54T',
 'N:Y298Y',
 'N:Y360Y',
 'N:Y86Y',
 'ORF10:C19C',
 'ORF10:Y26Y',
 'ORF1ab:A1151V',
 'ORF1ab:A1283V',
 'ORF1ab:A1298V',
 

In [13]:
# gisaid.columns

/home/al/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning:

Columns (37,38,40,41,42,43,44,45) have mixed types.Specify dtype option on import or set low_memory=False.



'England/205141731/2020', 'Lebanon/LAU-uk4/2020', 'England/204590625/2020', 
'England/204590575/2020', 'England/NORW-F6D33/2020'

In [16]:
g = gisaid.groupby(['pangolin_lineage', 'strain']).agg(mutations=('mutation', 'unique')).reset_index()
def check_mutation(x, mutation='S:N501Y'):
    if mutation in x:
        return True
    return False

g['501Y'] = g['mutations'].apply(check_mutation)

In [31]:
m = "S:D614G"
g['614G'] = g['mutations'].apply(check_mutation, args=(m,))

In [32]:
soi = g.loc[(g['614G']==False) 
          & (g['pangolin_lineage']=='B.1.1.7'), 
            'strain'].unique().tolist() + [input_params['patient_zero']]

In [33]:
soi

['England/CAMC-CFDB18/2020', 'NC_045512.2']

In [26]:
bs.fetch_seqs(input_params['msa_fp'], 'test_data/b117_wo_N501Y.fasta', soi, is_aligned=True)

1

In [185]:
# gisaid[(gisaid['pangolin_lineage']=='B.1.1.7')]['location'].value_counts()

In [186]:
# df

In [187]:
results['county_map'] = fig

In [143]:
gisaid.loc[(gisaid['codon_num']==452) & (gisaid['gene']=='S') & (gisaid['alt_aa']=='R'), 'strain'].unique().shape

(404,)

In [24]:
gisaid.loc[(gisaid['codon_num']==452) & (gisaid['gene']=='S') & (gisaid['alt_aa']=='R'), 'date'].min()

'2020-03-17'

In [ ]:
gisaid[gisaid['pangolin_lineage']=='B.1.1.70']

In [15]:
dists.loc[(dists['group']=='outgroup') & (dists['genetic_distance']>0.0008), 'pangolin_lineage'].value_counts()

B.1.1.70     48
B.1.351      19
B.1.160       1
B.1.1.31      1
B.1.160.6     1
B.1.1.1       1
B.1.367       1
C.14          1
Name: pangolin_lineage, dtype: int64